In [1]:
using JLD2
using Printf

In [7]:
function extract_from_config_table(config_filepath, key, run_idx)
    config_file = open(config_filepath, "r")
    header = readline(config_file)
    col_idx = findfirst(item -> cmp(item, key) == 0, split(header, " "))
    for line=readlines(config_file)
        entries = split(line, " ")
        if parse(Int, entries[1]) == run_idx
            close(config_file)
            return parse(Float64, entries[col_idx])
        end
    end
    close(config_file)
    return -1
end

function generate_table_data(buffer, datum; format_string="%s", header=false)
    elem_type = header ? "th" : "td"

    format = Printf.Format(format_string)
    print(buffer, @sprintf("<%s>", elem_type))
    print(buffer, Printf.format(format, datum))
    print(buffer, @sprintf("</%s>", elem_type))
end

function generate_table_row(buffer, data_list; format_strings=nothing, header=false)
    print(buffer, "<tr>")
    for (i, data)=enumerate(data_list)
        if !isnothing(format_strings)
            generate_table_data(buffer, data; format_string=format_strings[i], header)
        else
            generate_table_data(buffer, data; header)
        end
    end
    print(buffer, "</tr>\n")
end

function extract_from_jld_file(file, key)
    return file[key]
end

function find_in_param_file(param_text, key, config_file, run_idx)
    regex = Regex("^\\s*$key\\s*=\\s*(?<value>\\S*).*\$", "m")
    m=match(regex, param_text)
    if occursin("parse", m[:value])
        return extract_from_config_table(config_file, key, run_idx)
    else
        return parse(Float64, m[:value])
    end
end

function extract_README(run_directory)
    README_file = @sprintf("%s/README.txt", run_directory)
    return strip(read(README_file, String))
end

function extract_SLURM_ID(run_directory)
    regex = r"/([0-9]+(/[0-9]+)?)/?$"
    m=match(regex, run_directory)
    return replace(m[1], "/" => "_")
end

function extract_run_idx(run_directory)
    regex = r"/[0-9]+/([0-9]+)/?$"
    m=match(regex, run_directory)
    if m !== nothing
        return parse(Int, m[1])
    else
        return 0
    end
end

function mean(series)
    return sum(series) / length(series)
end

function extract_rsw_data(ic_file, param_file, run_directory, plot_data_file, idx)

    run_idx = extract_run_idx(run_directory)
    config_file = @sprintf("%s/froude-parameters.txt", run_directory)
    param_text = read(param_file, String)
    
    extraction_table = [
        ("Run",          "<b>%d</b>",     (idx) -> idx),
        ("SLURM_ID",     "%s",     (idx) -> extract_SLURM_ID(run_directory)),
        #("directory",    "%s",     (idx) -> run_directory),
        ("nx",           "%d"    , (idx) -> extract_from_jld_file(ic_file, "grid/nx")),
        ("Lx",           "%0.0fπ", (idx) -> extract_from_jld_file(ic_file, "grid/Lx")/π),
        ("Fr",           "%0.2f",  (idx) -> mean(extract_from_jld_file(plot_data_file, "computed/u_rms")) / sqrt(extract_from_jld_file(ic_file, "params/Cg2"))),
        ("f0",           "%0.2f",  (idx) -> extract_from_jld_file(ic_file, "params/f")),
        ("gH",           "%0.2f",  (idx) -> extract_from_jld_file(ic_file, "params/Cg2")),
        ("ν",            "%0.2e",  (idx) -> extract_from_jld_file(ic_file, "params/ν")),
        ("nν",           "%d",     (idx) -> extract_from_jld_file(ic_file, "params/nν")),
        ("ag",           "%0.2f",  (idx) -> find_in_param_file(param_text, "ag", config_file, run_idx)),
        ("aw",           "%0.2f",  (idx) -> find_in_param_file(param_text, "aw", config_file, run_idx)),
        ("Spin-up time", "%0.1f",  (idx) -> find_in_param_file(param_text, "spinup_T", config_file, run_idx)),
        ("Sim. time",    "%0.1f",  (idx) -> find_in_param_file(param_text, "T", config_file, run_idx)),
        ("README",       "%s",     (idx) -> extract_README(run_directory)),
        ("Figures",      "%s",     (idx) -> @sprintf("<a href=\"https://cims.nyu.edu/~nad9961/raytracing/%s_figures.html\">link</a>", extract_SLURM_ID(run_directory))),
    ]

    
    names = collect(map(x   -> x[1], extraction_table))
    format = collect(map(x  -> x[2], extraction_table))
    values = collect(map(x -> x[3](idx), extraction_table))
    return names, format, values
end

function generate_table!(buffer, rsw_config_file)
    is_header_generated = false
    for (idx,rsw_run)=enumerate(readlines(rsw_config_file))
        directory = split(rsw_run, " ")[2]
        ic_file = jldopen(@sprintf("%s/rsw.000000.jld2", directory), "r")
        param_file = @sprintf("%s/Parameters.jl", directory)
        plot_data_file = jldopen(@sprintf("%s/plot_data.jld2", directory))
        
        names, format_strings, values = extract_rsw_data(ic_file, param_file, directory, plot_data_file, idx)
        if !is_header_generated
            generate_table_row(buffer, names; header=true)
            is_header_generated = true
        end
        generate_table_row(buffer, values; format_strings)
        close(ic_file)
        close(plot_data_file)
    end
    return
end

table_contents_buffer = IOBuffer()
rsw_config_file = "/home/nad9961/JuliaRaytracingSW/analysis/Notebooks/rsw_suite/rsw-runs.txt"
generate_table!(table_contents_buffer, rsw_config_file)
table_contents = String(take!(table_contents_buffer))

LoadError: KeyError: key "computed" not found

In [3]:
function replace_template_content(template_file, output_buffer, var_table)
    for line=readlines(template_file)
        token_regex = r"(?:\$([a-zA-Z0-9_]+))+"
        m = match(token_regex, line)
        if m !== nothing
            for token=m.captures
                if haskey(var_table, token)
                    line = replace(line, ("\$"*token) => var_table[token])
                else
                    println("Token ", token, " not found")
                end
            end
        end
        print(output_buffer, line)
        print(output_buffer, "\n")
    end
end

output_buffer = open("raytracing/index.html", "w")
var_table = Dict("TABLE_CONTENT" => table_contents)
replace_template_content("/home/nad9961/JuliaRaytracingSW/visualization/index_template.html", output_buffer, var_table)
close(output_buffer)

In [4]:
function generate_image_pages!(rsw_config_file)
    for rsw_run=readlines(rsw_config_file)
        run_directory = split(rsw_run, " ")[2]
        slurm_id = extract_SLURM_ID(run_directory)
        output_buffer = open(@sprintf("raytracing/%s_figures.html", slurm_id), "w")
        
        var_table = Dict(
            "DIAG_TOTAL" => @sprintf("images/%s_energy_total.png", slurm_id),
            "DIAG_KE_PE" => @sprintf("images/%s_energy_KE_PE.png", slurm_id),
            "DIAG_GEO"   => @sprintf("images/%s_energy_geo.png", slurm_id),
            "DIAG_WAVE"  => @sprintf("images/%s_energy_wave.png", slurm_id),
            
            "RADIAL_INITIAL" => @sprintf("images/%s_energy_radial_initial.png", slurm_id),
            "RADIAL_AVERAGE" => @sprintf("images/%s_energy_radial_average.png", slurm_id),
            "RADIAL_FINAL"   => @sprintf("images/%s_energy_radial_final.png", slurm_id),
            
            "FLUX_TOTAL"      => @sprintf("images/%s_flux_total.png", slurm_id),
            "FLUX_COMPONENTS" => @sprintf("images/%s_flux_components.png", slurm_id))

        replace_template_content("/home/nad9961/JuliaRaytracingSW/visualization/figure_template.html", output_buffer, var_table)
        close(output_buffer)
    end
end

generate_image_pages!("/home/nad9961/JuliaRaytracingSW/analysis/Notebooks/rsw_suite/rsw-runs.txt")

In [7]:
map(x -> x, enumerate([4,5,6]))

3-element Vector{Tuple{Int64, Int64}}:
 (1, 4)
 (2, 5)
 (3, 6)